# V3 --- Re Fine-Tuning Model with Adjusted Tokenizer

This notebook contains a reimplementation of the fine tuning of distilbert for NER with the adjustment of a tokenizer that has been trained on both the conll2003 and ONLY the toxic comments from the wikipedia toxic comments corpus. The latter specification is what differs the V3 NER model from the V2. 

The proposed benefit of training the tokenizer on the conll2003 and the toxic comments from the wiki set is that there will be a much higher ratio of toxic tokens to conll2003 tokens. This means the tokenizer will be more likely to accept toxic words into its vocabulary, as opposed to subword tokenizing them. Since we are training this model for the process of labeling toxic entities, it is very important that the toxic entities remain whole tokens as much as possible.

## Understanding this Implementation

For a more detailed explanation of how the model, task, and training metric works, visit: ./NER_standard_model/V1_model_base_tokenizer/sagemaker_training_env.ipynb

The tokenizer was trained in ./NER_standard_model/tokenizer_training/labeler_tokenizer_training.ipynb

The tokenizer vocab file is at ./NER_standard_model/tokenizer_training/tokenizer_files/conll_wiki_tokenizer-vocab.txt


# Sagemaker Env


In [1]:
import sagemaker
sess = sagemaker.Session() #this creates a sagemaker session -
role = sagemaker.get_execution_role() #this gets permissions from the env where 
                                      #it is running. I am running in a sagemaker notebook instance

# HuggingFace Estimator
##### The huggingface estimator is a tool that will create a docker image of our specified hyperparams and conduct the training specified within the train.py training script.

In [2]:
from sagemaker.huggingface import HuggingFace


# hyperparameters which are passed to the training job
hyperparameters={'epochs': 3,
                 'per_device_train_batch_size': 16,
                 'per_device_eval_batch_size': 16
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='/home/ec2-user/SageMaker/NER_training_sagemaker/tox-wiki-addition-training', #This is the only change made to the doc
        instance_type='ml.g4dn.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.28.1',
        pytorch_version='2.0.0',
        py_version='py310',
        hyperparameters = hyperparameters
)

In [4]:
huggingface_estimator.fit({
    'train': "s3://conll2003-task/tokenized_conll2003/train/", 
    'test': "s3://conll2003-task/tokenized_conll2003/validation/"
    })

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-09-02-15-01-27-400


Using provided s3_resource
2023-09-02 15:01:27 Starting - Starting the training job...
2023-09-02 15:01:41 Starting - Preparing the instances for training......
2023-09-02 15:02:56 Downloading - Downloading input data
2023-09-02 15:02:56 Training - Downloading the training image.............................................
2023-09-02 15:10:29 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-02 15:10:36,640 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-02 15:10:36,658 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-02 15:10:36,667 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-09-02 15:10:36,674 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-09-02 15:10:38,668 sagemake

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1713: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
11%|█         | 142/1317 [00:19<02:29,  7.89it/s]


22%|██▏       | 286/1317 [00:39<02:29,  6.88it/s]


33%|███▎      | 434/1317 [00:59<01:58,  7.44it/s]


...

22%|██▏       | 11/51 [00:00<00:03, 12.39it/s]#033[A


98%|█████████▊| 50/51 [00:04<00:00,  8.99it/s]#033[A
#033[A
{'eval_loss': 0.2935323417186737, 'eval_precision': 0.6621699749470032, 'eval_recall': 0.616544051677732, 'eval_f1': 0.6385430217431703, 'eval_accuracy': 0.9107700005886855, 'eval_runtime': 6.5936, 'eval_samples_per_second': 492.902, 'eval_steps_per_second': 7.735, 'epoch': 2.0}
#015100%|██████████| 51/51 [00:06<00:00,  8.99it/s]#033[A
#015                                               #033[A
76%|███████▌  | 1000/1317 [02:31<00:45,  6.89it/s]
{'loss': 0.3056, 'learning_rate': 1.9400244798041617e-05, 'epoch': 2.28}
79%|███████▉  | 1046/1317 [02:39<00:39,  6.87it/s]


90%|████████▉ | 1185/1317 [02:59<00:17,  7.62it/s]


98%|█████████▊| 50/51 [00:05<00:00,  8.76it/s]#033[A
#033[A
{'eval_loss': 0.24818406999111176, 'eval_precision': 0.700082019502415, 'eval_recall': 0.6892158621927149, 'eval_f1': 0.6946064469460645, 'eval_accuracy': 0.9254135515394125, 'eval_runtime': 6.6869, 'eval_samples_per_second': 486.026, 'eval_steps_per_second': 7.627, 'epoch': 3.0}
#015100%|██████████| 51/51 [00:06<00:00,  8.76it/s]#033[A
#015                                               #033[A
{'train_runtime': 204.9251, 'train_samples_per_second': 205.553, 'train_steps_per_second': 6.427, 'train_loss': 0.46218288424527365, 'epoch': 3.0}
82%|████████▏ | 42/51 [00:04<00:01,  8.27it/s]



2023-09-02 15:14:30 Uploading - Uploading generated training model86%|████████▋ | 44/51 [00:04<00:00,  9.09it/s]
100%|██████████| 51/51 [00:06<00:00,  7.51it/s]
***** Eval results *****
2023-09-02 15:14:23,748 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2023-09-02 15:14:23,748 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2023-09-02 15:14:23,749 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-09-02 15:16:10 Completed - Training job completed
Training seconds: 814
Billable seconds: 814
